In [1]:
import tensorflow as tf
import numpy as np
import datetime, time

In [60]:
def genObs(data_length, full_length, dict_size=3):
    base = np.random.choice(range(dict_size), size=data_length)
    pad_length = full_length - 2*data_length - 1
    return np.concatenate([1 + base, np.zeros(1), 1 + np.array(list(reversed(base))), np.zeros(pad_length)])

def getMeta(x):
    mask = np.ones(x.shape)
    s = np.argmin(x) + 1
    mask[:s] = 0
    return mask.astype(np.int32)

def genSample(num, full_length=20):
    min_dlength = 1
    max_dlength = (full_length - 1) // 2 + 1
    x = [genObs(np.random.randint(min_dlength, max_dlength), full_length=full_length) for _ in range(num)]
    m = [getMeta(y) for y in x]
    return np.array(x), np.array(m)

def randomBatch(tensorTuple, batchSize=64):
    ids = np.random.choice(range(tensorTuple[0].shape[0]), batchSize)
    return (x[ids,] for x in tensorTuple)

def shuffleBatches(tensorTuple, batchSize=64):
    if type(tensorTuple) is list or type(tensorTuple) is tuple: 
        ids = list(range(tensorTuple[0].shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield (np.array(x[ids[i:lst],]) for x in tensorTuple)
    else:
        ids = list(range(tensorTuple.shape[0]))
        np.random.shuffle(ids)
        for i in range(0,len(ids),batchSize):
            lst = min(len(ids), i + batchSize)
            yield np.array(tensorTuple[ids[i:lst],])

In [61]:
train_x, train_m = genSample(100000, full_length=14)
valid_x, valid_m = genSample(10000, full_length=14)
valid_x[:10]

array([[ 2.,  3.,  2.,  1.,  3.,  0.,  3.,  1.,  2.,  3.,  2.,  0.,  0.,
         0.],
       [ 3.,  2.,  3.,  3.,  3.,  1.,  0.,  1.,  3.,  3.,  3.,  2.,  3.,
         0.],
       [ 3.,  3.,  3.,  0.,  3.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 3.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 2.,  1.,  2.,  1.,  2.,  1.,  0.,  1.,  2.,  1.,  2.,  1.,  2.,
         0.],
       [ 3.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 3.,  1.,  1.,  0.,  1.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 2.,  1.,  3.,  1.,  0.,  1.,  3.,  1.,  2.,  0.,  0.,  0.,  0.,
         0.],
       [ 2.,  2.,  3.,  1.,  2.,  3.,  0.,  3.,  2.,  1.,  3.,  2.,  2.,
         0.]])

In [62]:
SEQ_LEN = 14
DICT_SIZE = 3
RNN_SIZE = [15]

tf.reset_default_graph()

InnerCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)
rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE], state_is_tuple=True)
#rnnCell = tf.nn.rnn_cell.BasicRNNCell(10, activation=tf.nn.elu)


tfi_x = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.int32)
tfi_m = tf.placeholder(shape=(None, SEQ_LEN), dtype=tf.int32)

tfX = tf.one_hot(tfi_x, DICT_SIZE + 1, dtype=tf.float32)
tfM = tf.cast(tfi_m[:,1:], dtype=tf.float32)
tfY = tfX[:,1:,:]

tfHS, _ = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32, time_major=False)
tfH = tfHS#[-1] #get latest layer in RNN

tfHOut0 = tf.layers.dense(tfH, DICT_SIZE + 1)
tfHOut = tfHOut0[:,:-1,:] #remove last forecast

tfHLoss0 = tf.nn.softmax_cross_entropy_with_logits(labels=tfY, logits=tfHOut) #should be like batch_size x SEQ_LEN-1
tfLoss = tf.reduce_mean(tfHLoss0 * tfM)
tfTrain = tf.train.AdamOptimizer(1e-3).minimize(tfLoss)

tfPredicted = tf.cast(tf.argmax(tfHOut, axis=2), dtype=tf.int32) * tfi_m[:,1:]
tfActual = tfi_x[:,1:] * tfi_m[:,1:]

tfAccuracyChar = tf.reduce_mean(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32))
tfAccuracyObs = tf.reduce_mean(tf.reduce_min(tf.cast(tf.equal(tfActual, tfPredicted), dtype=tf.float32), axis=1))

tfsLoss = tf.summary.scalar('RMSE', tfLoss)
tfsAccuracy = tf.summary.scalar('Accuracy', 1-tfAccuracyObs)
tfsAll = tf.summary.merge([tfsLoss, tfsAccuracy])
tfsSaver = tf.train.Saver()

tffw = tf.summary.FileWriter('D:/Jupyter/Logs/00_A', tf.get_default_graph())
print('Graph creation complete')

Graph creation complete


In [64]:
dt_now = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
#tffw = tf.summary.FileWriter('D:/Jupyter/Logs/12RNN07-{0}'.format(dt_now), tf.get_default_graph())

batch_size = 1000
num_steps  = 10
num_epochs = 100
checkpoints = 5 #every 5 epochs save the model

fmtstr = 'Epoch {0} ({1:1.3} sec): \t\tVL:{2:1.3f}\t\tAC:{3:1.3f} ({4:1.3f})'
valid_batch = {tfi_x: valid_x, tfi_m: valid_m}
with tf.Session() as tfs:
    tfs.run(tf.global_variables_initializer())
    for i in range(num_epochs):
        te0 = time.perf_counter()
        for (mini_x, mini_m) in shuffleBatches((train_x, train_m), batchSize=batch_size):
            train_batch = {tfi_x:mini_x, tfi_m: mini_m}
            
            #l0 = tfLoss.eval(feed_dict=train_batch)
            #t0 = time.perf_counter()
            for j in range(num_steps):
                tfTrain.run(feed_dict=train_batch)
            #t1 = time.perf_counter()
            l1 = tfLoss.eval(feed_dict=train_batch)
    
        te1 = time.perf_counter()
        [lv,ac,ac0,summary] = tfs.run([tfLoss, tfAccuracyObs, tfAccuracyChar, tfsAll], feed_dict=valid_batch)
            #tffw.add_summary(summary, i)
        if i%checkpoints == 0 and i > 0:
            p = tfsSaver.save(tfs, 'D:/Jupyter/mltest/Models-13RNN02/model-{0:02d}.ckpt'.format(i))
            print('Model saved at checkpoint: {0}'.format(p))
                             
        print(fmtstr.format(i,te1-te0,lv,ac,ac0))
    valid_r = tfs.run(tfOutR, feed_dict=valid_batch)

Epoch 0 (34.9 sec): 		VL:0.230		AC:0.297 (0.897)
Epoch 1 (40.8 sec): 		VL:0.144		AC:0.485 (0.942)
Epoch 2 (35.0 sec): 		VL:0.103		AC:0.600 (0.959)
Epoch 3 (35.3 sec): 		VL:0.075		AC:0.714 (0.972)
Epoch 4 (34.6 sec): 		VL:0.057		AC:0.821 (0.983)
Model saved at checkpoint: D:/Jupyter/mltest/Models-13RNN02/model-05.ckpt
Epoch 5 (36.3 sec): 		VL:0.044		AC:0.855 (0.987)
Epoch 6 (38.9 sec): 		VL:0.031		AC:0.920 (0.993)
Epoch 7 (33.3 sec): 		VL:0.024		AC:0.941 (0.995)
Epoch 8 (31.6 sec): 		VL:0.023		AC:0.937 (0.995)
Epoch 9 (34.2 sec): 		VL:0.015		AC:0.970 (0.998)
Model saved at checkpoint: D:/Jupyter/mltest/Models-13RNN02/model-10.ckpt
Epoch 10 (34.6 sec): 		VL:0.013		AC:0.979 (0.998)
Epoch 11 (31.8 sec): 		VL:0.011		AC:0.984 (0.999)
Epoch 12 (33.0 sec): 		VL:0.009		AC:0.986 (0.999)
Epoch 13 (39.8 sec): 		VL:0.008		AC:0.991 (0.999)
Epoch 14 (32.9 sec): 		VL:0.008		AC:0.990 (0.999)
Model saved at checkpoint: D:/Jupyter/mltest/Models-13RNN02/model-15.ckpt
Epoch 15 (31.5 sec): 		VL:0.007		AC:0.

KeyboardInterrupt: 

In [47]:
tmp[1], getMeta(tmp[1])

(array([ 2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]))

In [78]:
DICT_SIZE = 3
RNN_SIZE = [15]

tf.reset_default_graph()

InnerCell = lambda n: tf.nn.rnn_cell.GRUCell(num_units=n, activation=tf.nn.elu)
rnnCell = tf.nn.rnn_cell.MultiRNNCell([InnerCell(s) for s in RNN_SIZE], state_is_tuple=True)
#rnnCell = tf.nn.rnn_cell.BasicRNNCell(10, activation=tf.nn.elu)


tfi_x = tf.placeholder(shape=(None, None), dtype=tf.int32)

tfX = tf.one_hot(tfi_x, DICT_SIZE + 1, dtype=tf.float32)
tfY = tfX[:,1:,:]

tfHS, _ = tf.nn.dynamic_rnn(rnnCell, inputs=tfX, dtype=tf.float32, time_major=False)
tfH = tfHS#[-1] #get latest layer in RNN

tfHOut0 = tf.layers.dense(tfH, DICT_SIZE + 1)
tfHOut = tfHOut0[:,:-1,:] #remove last forecast
tfPredicted = tf.cast(tf.argmax(tfHOut, axis=2), dtype=tf.int32)
tfsSaver = tf.train.Saver()
print('Graph creation complete')

model_id = 45
test_x = np.array([[3,1,1,0,0,0,0,0,0,0,0]])

with tf.Session() as tfs:
    tfsSaver.restore(tfs, 'D:/Jupyter/mltest/Models-13RNN02/model-{0:02d}.ckpt'.format(model_id))
    print(tfPredicted.eval(feed_dict={tfi_x:test_x}))

Graph creation complete
INFO:tensorflow:Restoring parameters from D:/Jupyter/mltest/Models-13RNN02/model-45.ckpt
[[3 1 1 1 1 0 0 0 0 0]]
